
# Cambios Recientes en los Códigos Postales de México (2020–2023)

## Actualizaciones a Nivel Nacional por Año

1. **Cambios en 2020:** No se encontró un número oficial públicamente divulgado sobre cuántos códigos postales se modificaron en 2020. Sin embargo, los datos disponibles sugieren que fue un año de pocos cambios netos en comparación con años posteriores. Dado que 2022 fue destacado como un año **récord** en cambios (ver abajo), se infiere que 2020 tuvo **considerablemente menos** modificaciones (probablemente solo algunos cientos de nuevos códigos) a nivel nacional. *Esto coincide con la relativa desaceleración en desarrollos urbanos durante la etapa inicial de la pandemia de COVID-19, lo cual habría limitado la creación de nuevos códigos postales en 2020.* (No hay una cifra exacta publicada en las fuentes conectadas para este año.)

2. **Cambios en 2022:** Fue el año con **mayor número de modificaciones de códigos postales** en México en tiempos recientes. De acuerdo con un análisis de GeoPostcodes, en 2022 se **introdujeron o cambiaron 837 códigos postales** a nivel nacional. Este pico refleja el **dinámico crecimiento urbano** y ajustes administrativos en distintas regiones del país durante ese año. En otras palabras, **837 nuevas zonas** recibieron códigos postales (o se actualizaron códigos existentes) solo en 2022, marcando un récord en la década reciente.

3. **Cambios en 2023:** Tampoco existe una cifra oficial publicada para 2023, pero las tendencias indican que el número de modificaciones fue **menor que el de 2022** (posiblemente en el rango de algunos pocos cientos de nuevos códigos). GeoPostcodes destacó únicamente a 2022 como el año de mayor actualización, lo que implica que **2023 no superó ese nivel**. Para tener contexto, el número total de códigos postales en México aumentó de alrededor de **\~33 mil** a inicios de la década a **más de 36 mil** hacia 2025, crecimiento que incluye el gran salto de 2022. Es razonable asumir que 2023 contribuyó con cierta porción de ese incremento (aunque menor que 2022). Las evidencias de solicitudes de nuevos códigos en 2023 corroboran que siguieron asignándose códigos en zonas de desarrollo, pero sin alcanzar la magnitud excepcional del año previo.

## Enfoque en la Zona del Valle de México (CDMX y Edomex)

La zona metropolitana del Valle de México —que abarca la **Ciudad de México** (CDMX) y el **Estado de México** (Edomex)— concentra una porción importante de los códigos postales del país. Actualmente, la Ciudad de México cuenta con **1,102 códigos postales**, mientras que el Estado de México posee **2,354 códigos**. Esto suma más de 3,400 códigos postales solo en la región metropolitana, reflejando su alta densidad de colonias y localidades.

Dado el intenso desarrollo urbano en el Valle de México, **muchos de los cambios de códigos postales ocurridos en 2020–2023 se concentraron en esta zona**. Por ejemplo, en 2023 se documentó una modificación en la Alcaldía **Xochimilco (CDMX)**, donde el código postal **16730 fue cambiado por el 13093** para actualizar la división postal de esa área. Asimismo, en octubre de 2023 se asignó un **nuevo código postal** para la zona de *Lomas de Tecamachalco* en el Estado de México, a solicitud de autoridades federales, con el fin de reconocer un inmueble y sector específico dentro de ese fraccionamiento. Estos casos ilustran cómo las **colonias en expansión o recién regularizadas** en la metrópoli recibieron códigos postales nuevos o ajustados en los años recientes.

En resumen, **2022 fue el año con más cambios** (837 nuevos códigos a nivel nacional), mientras que **2020 y 2023 tuvieron menos modificaciones** cada uno. Gran parte de estos cambios —especialmente las asignaciones de nuevos códigos— ocurrieron en zonas de crecimiento urbano como la **Ciudad de México y el Estado de México**, asegurando que la infraestructura postal se mantenga al día con la expansión de la mancha urbana en el Valle de México. Las cifras reflejan la importancia de mantener actualizado el *Catálogo Nacional de Códigos Postales* para **servir adecuadamente a las áreas urbanas en constante evolución**, evitando confusiones en la logística de correos y paquetería.

**Fuentes:** GeoPostcodes (actualizaciones globales de códigos postales, 2024); Portal oficial *codigo-postal.co* (estadísticas de códigos por estado); Solicitudes de asignación de CP (Transparencia SEPOMEX, 2023).


In [24]:
# Recarga el módulo
import importlib
import load_Data
import pandas as pd
importlib.reload(load_Data)
from load_Data import LoadData
from fuzzywuzzy import fuzz
import pickle
from typing import Dict, Tuple
import unicodedata
import re



In [25]:
loader = LoadData("../")

In [26]:
# Cargar datasets originales
original_datasets = loader.load_original_datasets()

# Cargar datasets clasificados donde los códigos postales son nulos
null_datasets = loader.load_classified_null_datasets()
equal_datasets = loader.load_classified_equal_datasets()
diff_datasets = loader.load_classified_different_datasets()

# Cargar datasets procesados
processed_datasets = loader.load_processed_datasets()

# Cargar GeoDataFrame con índice de marginación
marginacion_gdf = loader.load_marginacion_gdf()

# Usar como DataFrame regular (sin geometrías)
marginacion_df = loader.load_marginacion_gdf(as_dataframe=True)

# Cargar catálogo de códigos postales
catalogo_cp = loader.load_postal_codes_catalog()

In [27]:
def rename_dataframe_columns(dataframes_dict):
    
    for name, df in dataframes_dict.items():
        df_renamed = df.rename(columns={
            'SUS_CP': 'CP',
            'postal_code_api': 'CP_INDEX',
            'SUS_COL': 'COLONIA',
            'SUS_DEL': 'MUNICIPIO',
            'NOM_ENT': 'ENTIDAD',
        })
        dataframes_dict[name] = df_renamed
    
    return dataframes_dict

In [28]:
# Aplicar renombramiento a todos los diccionarios
original_datasets = rename_dataframe_columns(original_datasets)
null_datasets = rename_dataframe_columns(null_datasets)
equal_datasets = rename_dataframe_columns(equal_datasets)
diff_datasets = rename_dataframe_columns(diff_datasets)


In [29]:
for dataset_name, df in processed_datasets.items():
     df_renamed = df.rename(columns={
            'SUS_CP': 'CP',
            'postal_code_catalogo': 'CP_INDEX',
            'SUS_COL': 'COLONIA',
            'SUS_DEL': 'MUNICIPIO',
            'NOM_ENT': 'ENTIDAD',
        })
     processed_datasets[dataset_name] = df_renamed

In [30]:
catalogo_cp = catalogo_cp.rename(columns={
    'd_codigo': 'CP',
    'd_asenta': 'COLONIA',
    'D_mnpio': 'MUNICIPIO',
    'd_estado': 'ENTIDAD',
})
    

In [31]:
marginacion_gdf = marginacion_gdf.rename(columns={
    'NOM_ENT': 'ENTIDAD',
    'NOM_MUN': 'MUNICIPIO'
})

In [32]:
# convierte en marginacion_gdf …
marginacion_gdf['CP'] = (
    marginacion_gdf['CP']
    .astype(int)
    .astype(str)
    .str.zfill(5)
)




In [33]:

def clean_and_pad(df):
    # — 1) CP: convertir a int → str
    df['CP'] = df['CP'].astype(int).astype(str)
    # — 2) CP_INDEX: 
    #     a) convertir a str
    #     b) quitar “.0” al final
    df['CP_INDEX'] = (
        df['CP_INDEX']
        .fillna('')                                  # → '' en vez de NaN
        .astype(str)
        .str.replace(r'\.0$', '', regex=True)        # limpia ".0"
    )
    
    # — 3) aplicar padding de un '0' a la izquierda solo cuando la longitud es 4
    for col in ['CP','CP_INDEX']:
        mask4 = df[col].str.len() == 4
        df.loc[mask4, col] = '0' + df.loc[mask4, col]
    
    return df

# aplicarlo a todos tus dataframes
for key, df in original_datasets.items():
    original_datasets[key] = clean_and_pad(df)




In [34]:
# Función para normalizar espacios en columnas de texto
def normalizar_espacios(df, columnas):
    """
    Normaliza espacios en las columnas especificadas:
    - Convierte múltiples espacios, tabulaciones y saltos de línea en un solo espacio
    - Elimina espacios al inicio y final del texto
    """
    for col in columnas:
        if col in df.columns:
            # Primero verificamos que sea tipo string/object
            if df[col].dtype == 'object':
                # Reemplazamos cualquier secuencia de espacios con un solo espacio
                df[col] = df[col].str.replace(r'\s+', ' ', regex=True)
                # Eliminamos espacios al inicio y final
                df[col] = df[col].str.strip()
    return df



In [35]:
# Aplicar la función a marginacion_gdf
marginacion_gdf = normalizar_espacios(marginacion_gdf, ['MUNICIPIO', 'COLONIA'])

In [36]:
for key, df in original_datasets.items():
    original_datasets[key] = normalizar_espacios(df, ['MUNICIPIO', 'COLONIA'])

In [51]:
marginacion_gdf['MUNICIPIO'].unique()

array(['azcapotzalco', 'coyoacan', 'cuajimalpa de morelos',
       'huixquilucan', 'alvaro obregon', 'gustavo a. madero', 'iztacalco',
       'iztapalapa', 'tlahuac', 'la magdalena contreras', 'milpa alta',
       'xochimilco', 'tlalpan', 'benito juarez', 'cuauhtemoc',
       'miguel hidalgo', 'naucalpan de juarez', 'venustiano carranza',
       'tizayuca', 'nopaltepec', 'acolman', 'amecameca', 'ayapango',
       'apaxco', 'atenco', 'atizapan de zaragoza', 'atlautla', 'axapusco',
       'coacalco de berriozabal', 'cocotitlan', 'coyotepec',
       'tepotzotlan', 'cuautitlan', 'cuautitlan izcalli', 'chalco',
       'tlalmanalco', 'chiautla', 'texcoco', 'chicoloapan', 'chiconcuac',
       'chimalhuacan', 'nezahualcoyotl', 'ecatepec de morelos', 'tecamac',
       'tlalnepantla de baz', 'ecatzingo', 'huehuetoca', 'hueypoxtla',
       'tequixquiac', 'isidro fabela', 'ixtapaluca', 'jaltenco',
       'jilotzingo', 'juchitepec', 'melchor ocampo', 'nextlalpan',
       'nicolas romero', 'otumba',

In [52]:


# 1) Concatenas todas las series “MUNICIPIO”
todas_las_series = pd.concat(
    [df['MUNICIPIO'].dropna() for df in original_datasets.values()],
    ignore_index=True
)

# 2) Obtienes los únicos
valores_unicos = todas_las_series.unique()

print(valores_unicos)


['nezahualcoyotl' 'chiautla' 'chalco' 'coyotepec' 'hueypoxtla' 'tecamac'
 'temascalapa' 'teotihuacan' 'tultitlan' 'huautla' 'texcoco' 'teoloyucan'
 'cuajimalpa de morelos' 'zacatlan' 'coacalco de berriozabal' 'papalotla'
 'tizayuca' 'chiconcuac' 'nicolas romero' 'jilotzingo' 'tultepec'
 'tlalnepantla de baz' 'acolman' 'tlalmanalco' 'zumpango' 'tepetlaoxtoc'
 'ozumba' 'ecatepec de morelos' 'alvaro obregon' 'cuautitlan izcalli'
 'tlalpan' 'tlahuac' 'cuauhtemoc' 'naucalpan de juarez' 'huehuetoca'
 'ocoyoacac' 'tianguistenco' 'xochimilco' 'tonanitla' 'gustavo a madero'
 'nextlalpan' 'queretaro' 'iztapalapa' 'atenco' 'cintalapa'
 'melchor ocampo' 'chapulhuacan' 'miguel hidalgo' 'la magdalena contreras'
 'iztacalco' 'chimalhuacan' 'azcapotzalco' 'monterrey'
 'santa lucia del camino' 'la paz' 'jaltenco' 'venustiano carranza'
 'coyoacan' 'cocotitlan' 'chapa de mota' 'benito juarez' 'tlatlaya'
 'solidaridad' 'valle de chalco solidaridad' 'atizapan de zaragoza'
 'ayapango' 'cuautlancingo' 'tuxtl

In [53]:
arr1.size

574

In [54]:
arr2.size

76

In [55]:
import numpy as np

arr1 = valores_unicos
arr2 = marginacion_gdf['MUNICIPIO'].dropna().unique()

comunes_np = np.intersect1d(arr1, arr2)
print(f"Hay {comunes_np.size} municipios en común:")
print(comunes_np)


Hay 74 municipios en común:
['acolman' 'alvaro obregon' 'amecameca' 'apaxco' 'atenco'
 'atizapan de zaragoza' 'atlautla' 'axapusco' 'ayapango' 'azcapotzalco'
 'benito juarez' 'chalco' 'chiautla' 'chiconcuac' 'chimalhuacan'
 'coacalco de berriozabal' 'cocotitlan' 'coyoacan' 'coyotepec'
 'cuajimalpa de morelos' 'cuauhtemoc' 'cuautitlan' 'cuautitlan izcalli'
 'ecatepec de morelos' 'ecatzingo' 'huehuetoca' 'hueypoxtla'
 'huixquilucan' 'isidro fabela' 'ixtapaluca' 'iztacalco' 'iztapalapa'
 'jaltenco' 'jilotzingo' 'juchitepec' 'la magdalena contreras' 'la paz'
 'melchor ocampo' 'miguel hidalgo' 'milpa alta' 'naucalpan de juarez'
 'nextlalpan' 'nezahualcoyotl' 'nicolas romero' 'nopaltepec' 'otumba'
 'ozumba' 'papalotla' 'san martin de las piramides' 'tecamac' 'temamatla'
 'temascalapa' 'tenango del aire' 'teoloyucan' 'teotihuacan'
 'tepetlaoxtoc' 'tepetlixpa' 'tepotzotlan' 'tequixquiac' 'texcoco'
 'tezoyuca' 'tizayuca' 'tlahuac' 'tlalmanalco' 'tlalnepantla de baz'
 'tlalpan' 'tonanitla' 'tult

In [56]:
# Encontrar los elementos que están en arr1 pero no en la intersección
elementos_exclusivos_arr1 = np.setdiff1d(arr1, comunes_np)

# Mostrar el resultado
print(f"Hay {elementos_exclusivos_arr1.size} municipios en arr1 que no están en marginacion_gdf:")
print(elementos_exclusivos_arr1)

Hay 500 municipios en arr1 que no están en marginacion_gdf:
['acajete' 'acambaro' 'acambay' 'acapulco de juarez' 'acatepec' 'acatlan'
 'acaxochitlan' 'acayucan' 'actopan' 'aculco' 'agua dulce'
 'aguascalientes' 'ahuazotepec' 'alfajayucan' 'allende'
 'almoloya de alquisiras' 'almoloya de juarez' 'almoloya del rio'
 'altotonga' 'amanalco' 'amatepec' 'amaxac de guerrero'
 'amealco de bonfil' 'amixtlan' 'amozoc' 'angangueo' 'angel r cabada'
 'apan' 'apaseo el alto' 'apaseo el grande' 'apetatitlan de a carbajal'
 'apizaco' 'apodaca' 'arcelia' 'arroyo seco' 'asuncion ixtaltepec'
 'asuncion nochixtlan' 'atitalaquia' 'atizapan' 'atlacomulco' 'atlapexco'
 'atlatlahucan' 'atlixco' 'atotonilco de tula' 'atzacan' 'atzitzintla'
 'ayahualulco' 'ayala' 'ayotlan' 'ayutla de los libres'
 'bahia de banderas' 'boca del rio' 'cacahoatan' 'cadada morelos'
 'calimaya' 'calnali' 'calpulalpan' 'canada morelos' 'capulhuac'
 'cardenas' 'carmen' 'castillo de teayo' 'cazones de herrera' 'celaya'
 'centro' 'chalch

In [57]:
catalogo_cp['MUNICIPIO'].unique().size

141

In [63]:
arr3 = catalogo_cp['MUNICIPIO'].dropna().unique()
arr4 = elementos_exclusivos_arr1
comunes_np2 = np.intersect1d(arr1, arr3)
print(f"Hay {comunes_np2.size} municipios en común:")
print(comunes_np2)


Hay 138 municipios en común:
['acolman' 'aculco' 'almoloya de alquisiras' 'almoloya de juarez'
 'almoloya del rio' 'alvaro obregon' 'amanalco' 'amatepec' 'amecameca'
 'apaxco' 'atenco' 'atizapan' 'atizapan de zaragoza' 'atlacomulco'
 'atlautla' 'axapusco' 'ayapango' 'azcapotzalco' 'benito juarez'
 'calimaya' 'capulhuac' 'chalco' 'chapa de mota' 'chapultepec' 'chiautla'
 'chiconcuac' 'chimalhuacan' 'coacalco de berriozabal' 'coatepec harinas'
 'cocotitlan' 'coyoacan' 'coyotepec' 'cuajimalpa de morelos' 'cuauhtemoc'
 'cuautitlan' 'cuautitlan izcalli' 'donato guerra' 'ecatepec de morelos'
 'ecatzingo' 'el oro' 'huehuetoca' 'hueypoxtla' 'huixquilucan'
 'isidro fabela' 'ixtapaluca' 'ixtapan de la sal' 'ixtapan del oro'
 'ixtlahuaca' 'iztacalco' 'iztapalapa' 'jaltenco' 'jilotepec' 'jilotzingo'
 'jiquipilco' 'jocotitlan' 'joquicingo' 'juchitepec'
 'la magdalena contreras' 'la paz' 'lerma' 'luvianos' 'malinalco'
 'melchor ocampo' 'metepec' 'mexicaltzingo' 'miguel hidalgo' 'milpa alta'
 'morelo

---

In [ ]:
for key, df in original_datasets.items():
    # Añadir columna GM_2020 con valores nulos
    df['GM_2020'] = ""
    # Añadir columna HASH_CP con strings vacíos
    df['HASH_CP'] = ""
    
    # Añadir columna geometry con valores nulos
    # Si trabajas con GeoPandas, podrías necesitar un tipo específico
    df['geometry'] = ""

    # Actualizar el DataFrame en el diccionario
    original_datasets[key] = df

In [ ]:
for dataset_name, df in original_datasets.items():
    print(f"Dataset: {dataset_name}")
    print(df.head())

Dataset: df_coordenadas_sustentantes_2020
                         COLONIA     CP       MUNICIPIO ENTIDAD  lat_centro  \
0  ampliacion general jose vicen  57710  nezahualcoyotl  edomex   19.384366   
1                 santa catarina  56030        chiautla  edomex   19.549580   
2              nueva san antonio  56605          chalco  edomex   19.277787   
3                       caltenco  54665       coyotepec  edomex   19.785197   
4                     hueypoxtla  55670      hueypoxtla  edomex   19.910035   

   lng_centro  lat_principal  lng_principal CP_INDEX GM_2020 HASH_CP geometry  
0  -99.009547      19.383591     -99.010130    57710                           
1  -98.880199      19.549059     -98.883151    56030                           
2  -98.892980      19.278850     -98.892805    56605                           
3  -99.199699      19.785052     -99.202366                                    
4  -99.076913      19.910934     -99.075907    55670                           
Dat

In [ ]:
def is_empty(valor) -> bool:
    """
    Devuelve True si el valor es NaN o una cadena vacía (solo espacios).
    """
    return pd.isna(valor) or (isinstance(valor, str) and valor.strip() == "")


In [ ]:
def normalize_text(texto) -> str:
    """
    Normaliza una cadena:
    - Convierte a minúsculas
    - Elimina acentos
    - Colapsa espacios, tabulaciones y saltos de línea
    """
    if pd.isna(texto):
        return ""
    s = str(texto).lower()
    # Descomponer y eliminar diacríticos
    s = unicodedata.normalize('NFKD', s)
    s = ''.join(c for c in s if not unicodedata.combining(c))
    # Colapsar cualquier whitespace en un solo espacio
    s = re.sub(r'\s+', ' ', s).strip()
    return s


In [ ]:






def aplicar_fuzzy_match(
    original_datasets: Dict[str, pd.DataFrame],
    marginacion_gdf: pd.DataFrame,
    umbral: int = 80
) -> Tuple[Dict[str, pd.DataFrame], Dict[str, pd.DataFrame]]:
    """
    Completa campos vacíos en cada DataFrame de original_datasets utilizando fuzzy matching
    contra marginacion_gdf. Solo coincide filas con ENTIDAD idéntica y similitud
    token_sort_ratio >= umbral en COLONIA y MUNICIPIO.

    Retorna dos diccionarios:
    - datasets_process_fuzz_full: DataFrames con al menos un valor actualizado.
    - datasets_process_fuzz_empty: DataFrames sin actualizaciones.
    """
    # Pre-normalizar marginación
    margin = marginacion_gdf.copy()
    margin['__COLONIA_NORM'] = margin['COLONIA'].apply(normalize_text)
    margin['__MUNICIPIO_NORM'] = margin['MUNICIPIO'].apply(normalize_text)
    # Agrupar por ENTIDAD
    grupos_entidad = {
        entidad: df.reset_index(drop=True)
        for entidad, df in margin.groupby('ENTIDAD')
    }

    full, empty = {}, {}

    for key, df in original_datasets.items():
        df_out = df.copy()
        # Normalizar campos de búsqueda
        df_out['__COLONIA_NORM'] = df_out['COLONIA'].apply(normalize_text)
        df_out['__MUNICIPIO_NORM'] = df_out['MUNICIPIO'].apply(normalize_text)
        actualizado = False

        for idx, row in df_out.iterrows():
            ent = row.get('ENTIDAD')
            # Solo si existe grupo para esta entidad
            if ent not in grupos_entidad:
                continue
            # Si todos los campos ya están llenos, saltar
            if not (is_empty(row.get('GM_2020')) or is_empty(row.get('HASH_CP')) or is_empty(row.get('geometry'))):
                continue

            colonia_norm = row['__COLONIA_NORM']
            municipio_norm = row['__MUNICIPIO_NORM']
            best_match = None
            best_score = -1

            # Iterar candidatos de la entidad
            for _, cand in grupos_entidad[ent].iterrows():
                col_sim = fuzz.token_sort_ratio(colonia_norm, cand['__COLONIA_NORM']) if colonia_norm and cand['__COLONIA_NORM'] else 0
                mun_sim = fuzz.token_sort_ratio(municipio_norm, cand['__MUNICIPIO_NORM']) if municipio_norm and cand['__MUNICIPIO_NORM'] else 0
                # Requerir ambos campos >= umbral
                if col_sim >= umbral and mun_sim >= umbral:
                    score = min(col_sim, mun_sim)
                else:
                    continue

                if score > best_score:
                    best_score = score
                    best_match = cand

            # Si hay match, rellenar campos vacíos
            if best_match is not None:
                if is_empty(row.get('GM_2020')):
                    df_out.at[idx, 'GM_2020'] = best_match['GM_2020']
                    actualizado = True
                if is_empty(row.get('HASH_CP')):
                    df_out.at[idx, 'HASH_CP'] = best_match['CP']
                    actualizado = True
                if is_empty(row.get('geometry')):
                    df_out.at[idx, 'geometry'] = best_match['geometry']
                    actualizado = True

        # Clasificar según actualizaciones
        # Eliminar columnas auxiliares antes de retornar
        df_final = df_out.drop(columns=['__COLONIA_NORM', '__MUNICIPIO_NORM'])
        if actualizado:
            full[key] = df_final
        else:
            empty[key] = df_final

    return full, empty


In [ ]:
datasets_full, datasets_empty = aplicar_fuzzy_match(
    original_datasets,
    marginacion_gdf,
    umbral=70
)
# Ver resultados
print(f"DataFrames con datos actualizados: {len(datasets_full)}")
print(f"DataFrames sin coincidencias: {len(datasets_empty)}")

KeyboardInterrupt: 